In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from reactit.reactit import ReactionsListGenerator
from reactit.reactit import MappingtoReaction

import tqdm
compounds = {0:'CO2',
              1:'H2O', 
              2:'SO2', 
              3:'NO2', 
              4:'H2S', 
              5:'CH4', 
              6:'CH3', 
              7:'C2H4',
              8:'N2',
              9:'O2',
              10:'H2',
              11:'CO',
              12:'S8',
              13:'H2SO4',
              14:'H2O2',
              15:'HNO3',
              16:'H2SO3',
              17:'HNO2',
              18:'C2H4',
              19:'NH4',
              20:'NH3',
              21:'SO3',
              22:'O3',
              23:'C2H6',
              24:'C3H8',
              25:'C2H5OH'}
#compounds = {0:'O2',1:'H2O',2:'H2'}

In [27]:
rlg = ReactionsListGenerator(no_compounds=len(compounds))
reactions = list(rlg.iterate(4))

In [28]:
mappings = MappingtoReaction(reactions = reactions, compounds = compounds)
final_reactions = mappings.run_all()

100%|██████████| 4090/4090 [00:13<00:00, 302.85it/s]


In [29]:
import pandas as pd 
pd.Series(final_reactions)

0              1 SO2 + 1 H2O2 = 1 H2SO4
1              1 CO2 + 3 CH4 = 2 C2H5OH
2                  12 O2 + 1 S8 = 8 SO3
3                  1 O2 + 1 H2 = 1 H2O2
4                  1 H2 + 2 NH3 = 2 NH4
                     ...               
984      4 CO2 + 1 H2S = 4 CO + 1 H2SO4
985    2 CO2 + 10 H2O = 7 H2O2 + 1 C2H6
986      5 N2 + 9 H2O2 = 6 HNO3 + 4 NH3
987      6 CO2 + 6 C2H6 = 9 C2H4 + 4 O3
988      3 CH4 + 2 O2 = 2 H2O2 + 1 C3H8
Length: 989, dtype: object

In [30]:
mappings = MappingtoReaction(reactions = reactions, compounds = compounds)
approved = mappings.remove_indices(reactions)
strings = mappings.convert_to_string(approved)

In [31]:
def mp_function(iterable):
    reactants = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[0]}
    products = {molecule:mappings.parse_molecule(molecule) for molecule in iterable[1]}
    balanced = mappings.balance_reaction(reactants,products)
    if balanced:
        return(balanced)
    #else:
    #    return(None)
    
import tqdm_pathos 

screened = tqdm_pathos.map(mp_function,strings)
screened = [x for x in screened if x is not None]

  0%|          | 0/4090 [00:00<?, ?it/s]

In [32]:
import pandas as pd 
pd.Series(screened)

0              1 SO2 + 1 H2O2 = 1 H2SO4
1              1 CO2 + 3 CH4 = 2 C2H5OH
2                  12 O2 + 1 S8 = 8 SO3
3                  1 O2 + 1 H2 = 1 H2O2
4                  1 H2 + 2 NH3 = 2 NH4
                     ...               
984      4 CO2 + 1 H2S = 4 CO + 1 H2SO4
985    2 CO2 + 10 H2O = 7 H2O2 + 1 C2H6
986      5 N2 + 9 H2O2 = 6 HNO3 + 4 NH3
987      6 CO2 + 6 C2H6 = 9 C2H4 + 4 O3
988      3 CH4 + 2 O2 = 2 H2O2 + 1 C3H8
Length: 989, dtype: object